In [1]:
from argparse import ArgumentParser, ArgumentDefaultsHelpFormatter
import numpy as np
import pickle as pkl
from tqdm import tqdm_notebook as tqdm

import sys
sys.path.insert(0,'../../')

from rcalgo.tf.tfmodels.transformer_lm import *
from rcalgo.tf.tftraining.train_data import *

from rcalgo.tf.utils.text_utils import *
from sklearn.model_selection import train_test_split

In [2]:
with open('tmp_photo_comments_sentences.pkl','rb') as f:
    sentences = pkl.load(f)
sentences = np.array([s for s in sentences if len(s) > 0])
seqlen = np.array([len(s) for s in sentences])

In [3]:
word_freq = word_count(sentences, word_seg=False)
term_id_map = build_word2idx(word_freq, freq_threshold=10)


In [4]:
from rcalgo.tf.utils.train_config import *

args = parse_initial_training_args(
    [
        '--learning-rate', '0.0001',
        '--n-epochs', '5',
        '--opt', 'adam',
        '--batch-size', '512',
        '--p-coef', '0',
        '--keep-prob','0.5',
        '--embedding-size', '512',
        '--model-dir', 'model/',
        '--model-name', 'xxx',
        '--num-layers', '2',
        '--gpu-id', '0',
        '--gpu-num', '4',
        '--maxlen', '100',
        '--epoch-save',
        '--n-head', '8',
        '--position-type','2,3,4,5',
        '--transformer-func','relu',
        '--conv-num-filters', '512',
        '--low-sent-scale-length', '400',
        '--high-sent-scale-length', '800',
        '--combine-mode', 'ADD',
        '--position-info',
        '--output-reduce-size', '256',
        '--expand-input',
    ]
)
#args.sampled_softmax = False


In [5]:
graph_to_use = tf.Graph()
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
#RNNLMModel, BiRNNLMModel, TransformerLMModel，MaskedTransformerLMModel(sentences, , True)
rnnlm_model = MaskedTransformerLMModel(args, graph_to_use, word_dict=term_id_map)
rnnlm_model.build_model()
rnnlm_model.build_model_summary()
display(rnnlm_model.model_summary())
#rnnlm_model.run([sentences, True], test_size=0.2)

W1224 11:58:09.431318 139726815381312 deprecation.py:323] From /home/web_server/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/lookup_ops.py:1152: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1224 11:58:09.813462 139726815381312 deprecation.py:506] From /home/web_server/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:507: calling count_nonzero (from tensorflow.python.ops.math_ops) with axis is deprecated and will be removed in a future version.
Instructions for updating:
reduction_indices is deprecated, use axis instead


total parameter number: 14898704


variable_name  variable_shape  \
0     gpu/embedding/embedding_layer/embedding_table:0     [5135, 512]   
1            gpu/mw_conv_block/conv_1d/he_uniform/W:0   [2, 512, 512]   
2                       gpu/mw_conv_block/conv_1d/B:0           [512]   
3          gpu/mw_conv_block/conv_1d_1/he_uniform/W:0   [3, 512, 512]   
4                     gpu/mw_conv_block/conv_1d_1/B:0           [512]   
5          gpu/mw_conv_block/conv_1d_2/he_uniform/W:0   [4, 512, 512]   
6                     gpu/mw_conv_block/conv_1d_2/B:0           [512]   
7          gpu/mw_conv_block/conv_1d_3/he_uniform/W:0   [5, 512, 512]   
8                     gpu/mw_conv_block/conv_1d_3/B:0           [512]   
9                 gpu/pos_embedding/embedding_table:0      [100, 512]   
10  gpu/transform2_block/multi_head_attention/conv...  [1, 512, 1536]   
11  gpu/transform2_block/multi_head_attention/conv...          [1536]   
12  gpu/transform2_block/multi_head_attention/conv...   [1, 512, 512]   
13  gpu/transform2_block/multi_head_attention/conv...           [512]   
14               gpu/transform2_block/ln_layer/beta:0           [512]   
15              gpu/transform2_block/ln_layer/gamma:0           [512]   
16  gpu/transform2_block/ffn_mlp/conv_1d/he_unifor...  [1, 512, 2048]   
17           gpu/transform2_block/ffn_mlp/conv_1d/B:0          [2048]   
18  gpu/transform2_block/ffn_mlp/conv_1d_1/he_unif...  [1, 2048, 512]   
19         gpu/transform2_block/ffn_mlp/conv_1d_1/B:0           [512]   
20             gpu/transform2_block/ln_layer_1/beta:0           [512]   
21            gpu/transform2_block/ln_layer_1/gamma:0           [512]   
22  gpu/transform2_block_1/multi_head_attention/co...  [1, 512, 1536]   
23  gpu/transform2_block_1/multi_head_attention/co...          [1536]   
24  gpu/transform2_block_1/multi_head_attention/co...   [1, 512, 512]   
25  gpu/transform2_block_1/multi_head_attention/co...           [512]   
26             gpu/transform2_block_1/ln_layer/beta:0           [512]   
27            gpu/transform2_block_1/ln_layer/gamma:0           [512]   
28  gpu/transform2_block_1/ffn_mlp/conv_1d/he_unif...  [1, 512, 2048]   
29         gpu/transform2_block_1/ffn_mlp/conv_1d/B:0          [2048]   
30  gpu/transform2_block_1/ffn_mlp/conv_1d_1/he_un...  [1, 2048, 512]   
31       gpu/transform2_block_1/ffn_mlp/conv_1d_1/B:0           [512]   
32           gpu/transform2_block_1/ln_layer_1/beta:0           [512]   
33          gpu/transform2_block_1/ln_layer_1/gamma:0           [512]   
34                         gpu/conv_1d/he_uniform/W:0   [1, 512, 256]   
35                                    gpu/conv_1d/B:0           [256]   
36                                gpu/ln_layer/beta:0           [256]   
37                               gpu/ln_layer/gamma:0           [256]   
38  gpu/transform2_block_2/multi_head_attention/co...   [1, 256, 768]   
39  gpu/transform2_block_2/multi_head_attention/co...           [768]   
40  gpu/transform2_block_2/multi_head_attention/co...   [1, 256, 256]   
41  gpu/transform2_block_2/multi_head_attention/co...           [256]   
42             gpu/transform2_block_2/ln_layer/beta:0           [256]   
43            gpu/transform2_block_2/ln_layer/gamma:0           [256]   
44  gpu/transform2_block_2/ffn_mlp/conv_1d/he_unif...  [1, 256, 1024]   
45         gpu/transform2_block_2/ffn_mlp/conv_1d/B:0          [1024]   
46  gpu/transform2_block_2/ffn_mlp/conv_1d_1/he_un...  [1, 1024, 256]   
47       gpu/transform2_block_2/ffn_mlp/conv_1d_1/B:0           [256]   
48           gpu/transform2_block_2/ln_layer_1/beta:0           [256]   
49          gpu/transform2_block_2/ln_layer_1/gamma:0           [256]   
50                          gpu/prediction/loss_0/W:0     [256, 5136]   
51                          gpu/prediction/loss_0/B:0          [5136]   

   parameters  
0     2629120  
1      524288  
2         512  
3      786432  
4         512  
5     1048576  
6         512  
7     1310720  
8         512  
9       51

In [6]:
with rnnlm_model.job.session.graph.as_default():
    #print(tf.get_collection('input_dict'))
    input_var  = tf.get_collection('input_dict')[0]
rnnlm_model.job.session.run([rnnlm_model.split_inputX[0]], 
                           feed_dict = {input_var:  ['12345678']})    

[array([[5133,   45,   48,  368,   46,   49,  178,  336,  375, 5134,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0]])]

In [7]:
def run_output(job, data, single_task_name=None, output_index=None, batch_size=1024):
    if isinstance(data, DataIterator):
        data_iter = data
    else:  # assert list
        data_iter = DataIterator(data, np.arange(
            len(data[0])), batch_size=batch_size)
    data_iter_tqdm = tqdm(data_iter, leave=False)
    task = job._get_tasks(single_task_name)[0]
    if output_index is not None and isinstance(output_index, int):
        output_index = [output_index]
    outputs = task.outputs if output_index is None else [
        task.outputs[idx] for idx in output_index]
    results = []
    for batch_index, batch in data_iter_tqdm:
        feed_dict = dict(zip(task.inputs, batch))
        output = job.session.run(outputs, feed_dict=feed_dict)
        results.append(output)
    return [np.concatenate([output[index] for output in results]) for index in range(len(outputs))]

In [8]:
results1 = rnnlm_model.job.session.run(rnnlm_model.tower_prediction_results, 
                                      feed_dict={rnnlm_model.job._get_tasks(None)[0].inputs[0]:sentences[0:10],
                                             #    rnnlm_model.job._get_tasks(None)[0].inputs[2]: [100,100]
                                                })
results2 = rnnlm_model.job.session.run(rnnlm_model.tower_prediction_results, 
                                       feed_dict={rnnlm_model.job._get_tasks(None)[0].inputs[0]:sentences[0:10],
                                             #    rnnlm_model.job._get_tasks(None)[0].inputs[2]: [100,100]
                                                })

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: Number of ways to split should evenly divide the split dimension, but got split_dim 0 (size = 10) and num_split 4
	 [[node input/split (defined at ../tfmodels/text_basic_model.py:128) ]]
  (1) Invalid argument: Number of ways to split should evenly divide the split dimension, but got split_dim 0 (size = 10) and num_split 4
	 [[node input/split (defined at ../tfmodels/text_basic_model.py:128) ]]
	 [[training/_113]]
0 successful operations.
3 derived errors ignored.

Errors may have originated from an input operation.
Input Source operations connected to node input/split:
 input/input_raw_sentence (defined at ../tfmodels/text_basic_model.py:78)

Input Source operations connected to node input/split:
 input/input_raw_sentence (defined at ../tfmodels/text_basic_model.py:78)

Original stack trace for 'input/split':
  File "/home/web_server/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", 

In [ ]:
rnnlm_model.job.session.run(rnnlm_model.output_dict['output'], 
                            feed_dict={rnnlm_model.job._get_tasks(None)[0].inputs[0]:sentences[0:10],
                                       #rnnlm_model.job._get_tasks(None)[0].inputs[1]:True,
                                       #rnnlm_model.job._get_tasks(None)[0].inputs[2]: [2],
                                       #rnnlm_model.job.is_training_op: True
                                                })

In [ ]:
results

In [ ]:
results = run_output(rnnlm_model.job, [sent], output_index=3, batch_size=1024)[0]
results = np.divide(results, np.linalg.norm(results, axis=1, keepdims=True))

In [ ]:
np.dot(results[0],results[1])

In [ ]:
rnnlm_model.indices_list[0]

In [ ]:
np.dot(results[0],results[1])

In [ ]:
#with open('../../experimental_code/faiss/rnn_sentences.pkl','wb') as f:
#    pkl.dump(sentences[0:1000000], f, protocol=4)
#    pkl.dump(results[0:1000000], f, protocol=4)